<a href="https://colab.research.google.com/github/smhall97/hallucinating_GANs/blob/main/Combined_CNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports

import os
import glob

import torch

import time
import copy

import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm



import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.optim import lr_scheduler

In [ ]:
# @title Helper Functions

def scale_minmax(X):

    X_scaled = (X - X.min()) / (X.max() - X.min())

    return X_scaled


def pickle_loader_mel(file):
  with open(file, 'rb') as f:
      data = pickle.load(f)
      data = np.transpose(data, axes=[1, 2, 0])
      data = scale_minmax(data)

      zeros = np.zeros(data.shape)

      data = np.concatenate((data, data, data), axis=2)

  return(data)


def pickle_loader_stft(file):
  with open(file, 'rb') as f:
      data = pickle.load(f)
      data = np.squeeze(data, axis=0)

      #real and imaginary parts are scaled independently
      data[:,:,0] = scale_minmax(data[:,:,0])
      data[:,:,1] = scale_minmax(data[:,:,1])

      zeros = np.zeros((data.shape[0],data.shape[1],1))
      data = np.concatenate((data, zeros), axis=2)

  return(data)


def make_sets(classes, items_per_class, ratios):
  """
  parameters:
  classes: number of classes in dataset
  items_per_class: elements per class (assumes that the dataset is balanced across classes)
  ratios: list or array with ratios for each subset [ratio_trainining, ratio_validation, ratio_test]
  """

  train_size = ratios[0] * items_per_class
  val_size = ratios[1] * items_per_class
  test_size = ratios[2] * items_per_class

  test_ix, val_ix, train_ix = np.array([]),np.array([]),np.array([])

  for i in range(classes): 
    class_ix = items_per_class * i
    
    train_ix = np.append(train_ix, np.arange(train_size) + class_ix)
    val_ix = np.append(val_ix, np.arange(train_size, train_size + val_size) + class_ix)
    test_ix = np.append(test_ix, np.arange(train_size + val_size, train_size + val_size + test_size) + class_ix)

  return train_ix.astype(int), val_ix.astype(int), test_ix.astype(int)

In [ ]:
# @title Set device (GPU or CPU)
# NMA code
# inform the user if the notebook uses GPU or CPU.

def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

device = set_device()

In [ ]:
mount_drive = False

if mount_drive:
  from google.colab import drive
drive.mount('/content/drive') #it will ask you for a verification code

# Set up experiment variables

In [ ]:
data_type = "stft" # alternative options "mel"
data_augmentations = "True" # if no augmentation "False"
augment_type = "shuffled" # alternative augmentations: 

dummy_mode = False

path = '/content/drive/MyDrive/HallucinatingGANs/Code/data/'

transform = 'mel'

n_fft = 1024
n_mels = 128
hop_length = 256 # smaller hop size leads to better reconstruction but takes longer to compute
power = 2.0 # squared power spectrogram
samplerate =  22050

def get_cfg_transform(t):
  if t == 'stft':
    params = '{}_{}'.format(str(n_fft), str(hop_length))
    pickle_loader = pickle_loader_stft
  
  elif t == 'mel':
    params = '{}_{}_{}'.format(str(n_fft), str(hop_length), n_mels)
    pickle_loader = pickle_loader_mel
  return params, pickle_loader

In [ ]:
# @title Load pretrained VGG
"""
code extracted from:
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#convnet-as-fixed-feature-extractor
https://pytorch.org/vision/stable/models.html

"""

vgg16 = models.vgg16(pretrained=True)

# Freeze the network except the last layer / unfreeze layers to allow finetuning
for param in vgg16.parameters():
    param.requires_grad = True # If True it will train

# Parameters of newly constructed modules have requires_grad=True by default
# Add on classifier

vgg16.classifier[6] = nn.Sequential(
                      nn.Linear(vgg16.classifier[3].in_features, 256),
                      nn.ReLU(), 
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
# @title Train model function from PyTorch

# Original code from this tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_acc_list, val_acc_list = [], []
    # train_loss, validation_loss = [], []
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            num_examples = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                num_examples += inputs.size(0)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            # Different to tutorial, hardcoded dataset size
            # print(dataset_sizes) from above 
            
            #if phase == 'train':
            #    num_examples = len(train_loader) * len(next(iter(train_loader))[0]) 
            #else:
            #    num_examples = len(val_loader) * len(next(iter(val_loader))[0])
            print('number of examples in loader = ', num_examples)
            print(f'RUNNING LOSS: {running_loss}, RUNNING CORRECTS: {running_corrects}')

            epoch_loss = running_loss / num_examples
            print()
            epoch_acc = running_corrects.double() / num_examples
            if phase == 'train':
              train_acc_list.append(epoch_acc)
            else:
              val_acc_list.append(epoch_acc)
          
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    # model = model.to('cuda')
    return model, train_acc_list, val_acc_list